In [1]:
# https://github.com/scipy/scipy/issues/7151
# https://apps.dtic.mil/sti/pdfs/AD1004183.pdf
# https://www.codeproject.com/Articles/21282/Compute-Permanent-of-a-Matrix-with-Ryser-s-Algorit

# https://rosettacode.org/wiki/Determinant_and_permanent
# https://codegolf.stackexchange.com/questions/97060/calculate-the-permanent-as-quickly-as-possible

# https://stackoverflow.com/questions/38738835/generating-gray-codes
# https://qiita.com/b1ueskydragon/items/75cfee42541ea723080c

# https://qiita.com/phdax/items/3064de264c7933bab2f5
# https://web.archive.org/web/20190108235115/https://www.hackersdelight.org/hdcodetxt/pop.c.txt
# http://graphics.stanford.edu/~seander/bithacks.html#CountBitsSetParallel
# https://stackoverflow.com/questions/9829578/fast-way-of-counting-non-zero-bits-in-positive-integer

# https://stackoverflow.com/questions/22227595/convert-integer-to-binary-array-with-suitable-padding

In [36]:
import numpy as np

from itertools import permutations
from operator import mul
from math import fsum
from functools import reduce

import scipy.special as spsp

import time

In [3]:
def slow_gray_code(n,dim):
    res = np.zeros(dim+1,dtype=np.int64)
    pos = dim - 1
    while n>0:
        res[pos] = n%2
        res[dim] += res[pos]
        n = n//2
        pos = pos-1
    return res

In [4]:
def count_bits(bit):
    count = bit
## 32 bits
#    count = (count & 0x55555555) + ((count >> 1) & 0x55555555)
#    count = (count & 0x33333333) + ((count >> 2) & 0x33333333)
#    count = (count & 0x0F0F0F0F) + ((count >> 4) & 0x0F0F0F0F)
#    count = (count & 0x00FF00FF) + ((count >> 8) & 0x00FF00FF)
#    count = (count & 0x0000FFFF) + ((count >>16) & 0x0000FFFF)
## 64 bits
    count = (count & 0x5555555555555555) + ((count & 0xAAAAAAAAAAAAAAAA) >> 1)
    count = (count & 0x3333333333333333) + ((count & 0xCCCCCCCCCCCCCCCC) >> 2)
    count = (count & 0x0F0F0F0F0F0F0F0F) + ((count & 0xF0F0F0F0F0F0F0F0) >> 4)
    count = (count & 0x00FF00FF00FF00FF) + ((count & 0xFF00FF00FF00FF00) >> 8)
    count = (count & 0x0000FFFF0000FFFF) + ((count & 0xFFFF0000FFFF0000) >> 16)
    count = (count & 0x00000000FFFFFFFF) + ((count & 0xFFFFFFFF00000000) >> 32)
    return count

def fast_gray_code(i,n):
    gray = i^(i>>1)
    return np.array(list(np.binary_repr(gray).zfill(n))+[count_bits(gray)]).astype(np.int64)

In [5]:
def calc_perm_slow(A,n):
    sum = 0
    rowsumprod = 0
    rowsum = 0
    chi = np.zeros(n+1,dtype=np.int64)
    C = 2**n
    for k in range(1,C):
        rowsumprod = 1
        chi = slow_gray_code(k,n)
        for m in range(n):
            rowsum = 0
            for p in range(n):
                #rowsum += chi[p] * A[m * n + p]
                rowsum += chi[p] * A[m,p]
            rowsumprod *= rowsum
        sum += (-1)**(n-chi[n]) * rowsumprod
    return sum

In [6]:
def prod(lst):
    return reduce(mul, lst, 1)

def perm(a):
    n = len(a)
    r = range(n)
    s = permutations(r)
    return fsum(prod(a[i][sigma[i]] for i in r) for sigma in s)

In [7]:
def npperm(M):
    n = M.shape[0]
    d = np.ones(n,dtype=np.float64)
    j =  0
    s = 1
    f = np.arange(n)
    v = M.sum(axis=0,dtype=np.float64)
    p = np.prod(v)
    while (j < n-1):
        v -= 2.0*d[j]*M[j]
        d[j] = -d[j]
        s = -s
        prod = np.prod(v)
        p += s*prod
        f[0] = 0
        f[j] = f[j+1]
        f[j+1] = j+1
        j = f[0]
    return p/2**(n-1) 

In [8]:
def calc_perm_fast(A,n):
    sum = 0
    rowsumprod = 0
    rowsum = 0
    chi = np.zeros(n+1,dtype=np.int64)
    C = 2**n
    for k in range(1,C):
        rowsumprod = 1
        chi = fast_gray_code(k,n)
        for m in range(n):
            rowsum = 0
            for p in range(n):
                #rowsum += chi[p] * A[m * n + p]
                rowsum += chi[p] * A[m,p]
            rowsumprod *= rowsum
        sum += (-1)**(n-chi[n]) * rowsumprod
    return sum

In [9]:
for a in (
        [
         [1, 2],
         [3, 4]],

        [
         [1, 2, 3, 4],
         [4, 5, 6, 7],
         [7, 8, 9, 10],
         [10, 11, 12, 13]],

        [
         [ 0,  1,  2,  3,  4],
         [ 5,  6,  7,  8,  9],
         [10, 11, 12, 13, 14],
         [15, 16, 17, 18, 19],
         [20, 21, 22, 23, 24]],
    ):
    print("")
    print(calc_perm_slow(np.array(a),np.array(a).shape[0]))
    print(calc_perm_fast(np.array(a),np.array(a).shape[0]))
    print(perm(a))
    print(npperm(np.array(a)))


10
10
10.0
10.0

29556
29556
29556.0
29556.0

6778800
6778800
6778800.0
6778800.0


In [39]:
L = 10
a = np.ones((L,L),dtype=np.float64)
print(a)

start = time.time()
print(calc_perm_slow(np.array(a),np.array(a).shape[0]))
end = time.time()
print("time:",end-start)

start = time.time()
print(calc_perm_fast(np.array(a),np.array(a).shape[0]))
end = time.time()
print("time:",end-start)

start = time.time()
print(perm(a))
end = time.time()
print("time:",end-start)

start = time.time()
print(npperm(np.array(a)))
end = time.time()
print("time:",end-start)

[[1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]]
3628800.0
time: 0.2358999252319336
3628800.0
time: 0.22943592071533203
3628800.0
time: 12.36816120147705
3628800.0
time: 0.004374980926513672


In [41]:
L = 14
a = np.ones((L,L),dtype=np.float64)
print(a)

start = time.time()
print(calc_perm_slow(np.array(a),np.array(a).shape[0]))
end = time.time()
print("time:",end-start)

start = time.time()
print(calc_perm_fast(np.array(a),np.array(a).shape[0]))
end = time.time()
print("time:",end-start)

#start = time.time()
#print(perm(a))
#end = time.time()
#print("time:",end-start)

start = time.time()
print(npperm(np.array(a)))
end = time.time()
print("time:",end-start)

[[1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]]
87178291200.0
time: 6.7552759647369385
87178291200.0
time: 6.939203262329102
87178291200.0
time: 0.06898903846740723


In [42]:
L = 20
a = np.ones((L,L),dtype=np.float64)
print(a)

#start = time.time()
#print(calc_perm_slow(np.array(a),np.array(a).shape[0]))
#end = time.time()
#print("time:",end-start)

#start = time.time()
#print(calc_perm_fast(np.array(a),np.array(a).shape[0]))
#end = time.time()
#print("time:",end-start)

#start = time.time()
#print(perm(a))
#end = time.time()
#print("time:",end-start)

start = time.time()
print(npperm(np.array(a)))
end = time.time()
print("time:",end-start)

[[1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.

In [43]:
L = 22
a = np.ones((L,L),dtype=np.float64)
print(a)

#start = time.time()
#print(calc_perm_slow(np.array(a),np.array(a).shape[0]))
#end = time.time()
#print("time:",end-start)

#start = time.time()
#print(calc_perm_fast(np.array(a),np.array(a).shape[0]))
#end = time.time()
#print("time:",end-start)

#start = time.time()
#print(perm(a))
#end = time.time()
#print("time:",end-start)

start = time.time()
print(npperm(np.array(a)))
end = time.time()
print("time:",end-start)

[[1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.

In [ ]:
L = 24
a = np.ones((L,L),dtype=np.float64)
print(a)

#start = time.time()
#print(calc_perm_slow(np.array(a),np.array(a).shape[0]))
#end = time.time()
#print("time:",end-start)

#start = time.time()
#print(calc_perm_fast(np.array(a),np.array(a).shape[0]))
#end = time.time()
#print("time:",end-start)

#start = time.time()
#print(perm(a))
#end = time.time()
#print("time:",end-start)

start = time.time()
print(npperm(np.array(a)))
end = time.time()
print("time:",end-start)

[[1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1. 1. 1. 1.

In [24]:
L = 100
#L = 4
L_A = 2
#L_A = 1
x = np.zeros((L,L), dtype = np.float64)
for j in range(L):
  for k in range(L):
    x[j,k] = np.sin((j+1)*(k+1)*np.pi / (L+1))
x *= np.sqrt(2.0 / (L+1))
eps = np.array([-2.0 * np.cos((n+1) * np.pi / (L+1)) for n in range(L)])

for j_t in range(19,20):
  time = 10.0#0.1 * j_t
  ## y by definition
  y = x @ (np.diag(np.exp(1j * eps * time))) @ x
  ## y for thermodynamic limit
  y_inf = np.zeros((L,L), dtype = np.complex128)
  for j in range(L):
    for k in range(L):
      # y_sub = - (-1j)**(j+k+2) * spsp.jn(j+k+2, 2.0 * time) \
      #         + (-1j)**(j-k  ) * spsp.jn(j-k  , 2.0 * time)
      y_sub = (-1j)**(j-k) * (spsp.jn(j-k, 2.0 * time) + (-1)**(k) * spsp.jn(j+k+2, 2.0 * time))
      y_inf[j,k] = y_sub
  ## compare
  # j = 3
  # for k in range(L):
  #   print(y[j,k], y_inf[j,k])
  #
  ## z by definition
  z = np.zeros((L,L), dtype = np.complex128)
  for j in range(L):
    for k in range(L):
      for m in range(L_A):
        z[j,k] += y[j,m].conjugate() * y[k,m]
  ## z for thermodynamic limit
  z_inf = np.zeros((L,L), dtype = np.complex128)
  for j in range(L):
    for k in range(L):
      for m in range(L_A):
        # z_inf[j,k] += y_inf[j,m].conjugate() * y_inf[k,m]
        z_inf[j,k] += (-1)**(k+1) * (+1j)**(j+k+2) * (spsp.jn(j-m, 2.0 * time) + (-1)**(m) * spsp.jn(j+m+2, 2.0 * time)) \
                                                   * (spsp.jn(k-m, 2.0 * time) + (-1)**(m) * spsp.jn(k+m+2, 2.0 * time))
  ## check
  j = 1
  for k in range(L):
    print(z[j,k], z_inf[j,k])

(9.973602639106635e-18+0.0009514789986535363j) 0.0009514789986535402j
(0.002349967707500385+0j) (0.002349967707500388+0j)
(9.52908356708066e-18+0.002114472645660744j) 0.0021144726456607463j
(0.005502579257198872+2.460199274078104e-17j) (0.005502579257198878+0j)
(2.084442360777736e-17+0.000677823928702337j) 0.0006778239287023604j
(0.008495695691574576-1.3651778003430206e-18j) (0.008495695691574582+0j)
(2.913551186672375e-17-0.005082141987386141j) -0.005082141987386129j
(0.006886841100052456-3.5086625732084797e-17j) (0.006886841100052475+0j)
(3.166092244488538e-17-0.012538989942277517j) -0.012538989942277522j
(-0.004339831123024822-4.203634537477256e-17j) (-0.004339831123024803+0j)
(-3.01485703461911e-17-0.009805890878175388j) -0.00980589087817541j
(-0.016680436370828656-2.1112867492479543e-17j) (-0.016680436370828666+0j)
(-4.9210387352020745e-17+0.010770604640388516j) 0.010770604640388492j
(-0.0030309615635082127+4.7502731821230387e-17j) (-0.00303096156350825+0j)
(8.671588377905204e-18+

In [25]:
print(z)

[[ 1.07304063e-03+0.00000000e+00j  9.97360264e-18-9.51478999e-04j
   2.90282697e-03+2.49347552e-17j ... -1.68754432e-17+1.36582706e-18j
   5.56138774e-19+7.34805268e-17j -5.31125440e-18-1.35048380e-17j]
 [ 9.97360264e-18+9.51478999e-04j  2.34996771e-03+0.00000000e+00j
   9.52908357e-18+2.11447265e-03j ...  3.30239027e-17+1.19086894e-18j
  -9.04649595e-17-2.01766465e-17j  1.24809413e-17+2.86032543e-17j]
 [ 2.90282697e-03-2.49347552e-17j  9.52908357e-18-2.11447265e-03j
   7.99300321e-03+0.00000000e+00j ... -5.05801010e-17+1.41385231e-17j
   7.80995608e-18+1.91061441e-16j -2.45305109e-17-3.63793918e-17j]
 ...
 [-1.68754432e-17-1.36582706e-18j  3.30239027e-17-1.19086894e-18j
  -5.05801010e-17-1.41385231e-17j ...  1.21848833e-30+0.00000000e+00j
  -7.12119260e-31-1.35467252e-30j  4.35113419e-31+9.70860679e-31j]
 [ 5.56138774e-19-7.34805268e-17j -9.04649595e-17+2.01766465e-17j
   7.80995608e-18-1.91061441e-16j ... -7.12119260e-31+1.35467252e-30j
   5.74102968e-30+0.00000000e+00j -1.39318227e-

In [26]:
print(np.linalg.det(z))

0j


In [27]:
print(np.linalg.det(x))

0.9999999999999742


In [28]:
print(np.linalg.det(x @ (np.diag(np.exp(1j * eps * time))) @ x))

(0.9999999999999638-8.206022995259741e-14j)


In [29]:
print(np.linalg.det(z_inf))

(-0+0j)
